In [2]:
import pymem
from pymem.pattern import pattern_scan_all



target_process = "javaw.exe"  # Define the name of the process
pm = pymem.Pymem(target_process)

In [ ]:
textpat=b"\\x00\\x00\\x00\\x00\\x00\\x00..\\x00\\x00..\\x00\\x00\\x64\\x04"

In [ ]:
# Pattern in byte format
pattern = b'\\x4\\x..\\x8B\\x..\\x28\\x45\\x8b\\x..\\x18\\x4\\x..\\x0F\\xBE\\x..\\x10\\x4\\x..\\x0F\\xB6\\x..\\x12\\x4\\x..\\x8B\\x..\\x24\\x45\\x0F\\xB6\\x..\\x11'
pattern=b'\\x8B\\x28\\x45\\x8b'
pattern=b'\\x..\\xbeK\\x10A\\x0f\\xb6'
pattern=b'\\x00..\\x8B\\x00..\\x28\\x45\\x8b\\x00..\\x18\\x00..\\x0F\\xBE\\x00..\\x10\\x00..\\x0F\\xB6\\x00..\\x12\\x00..\\x8B\\x00..\\x24\\x45\\x0F\\xB6\\x00..\\x11'
pattern=b'\\x12\\x8B\\x24\\x45\\x0F\\xB6\\x11'
pattern=b'\x18\x45\x0F.\x4B\x10\x41\x0F\xB6'
pattern=b'\\x8B.\\x28\\x45\\x8b.\\x18.\\x0F\\xBE.\\x10.\\x0F\\xB6.\\x12.\\x8B.\\x24'#24后面有问题
pattern=b'\\x8B.\\x28\\x45\\x8b.\\x18.\\x0F\\xBE.\\x10.\\x0F\\xB6.\\x12'#有点用，不是很好用

# Get all addresses with the given pattern
aob_address_list = pymem.pattern.pattern_scan_all(
    handle=pm.process_handle, 
    pattern=pattern,
    return_multiple=True
)

# Printing found addresses
print(aob_address_list)
for i in aob_address_list:
    print(hex(i+3))

In [ ]:
hex(aob_address_list[0])

In [ ]:
data = pm.read_bytes(1867983663273+7, 5)
print(data)
#byte to int
print()
for i in range(0, 5):
    print(hex(data[i]))
# print(hex(int.from_bytes(data, byteorder='little', signed=True)))
# byte_array = bytes.fromhex()  # 将十六进制字符串转换为字节数组
# byte_list = list(byte_array)  # 将字节数组转换为列表
# byte_list
# byte_array = bytes.fromhex(str(hex(data[i])))  # 将十六进制字符串转换为字节数组
# byte_list = list(byte_array)  # 将字节数组转换为列表

# print(byte_list)  # 输出结果
bytes_list = [bytes([b]) for b in data]

hex_list = [hex(b)[2:].upper() for b in data]


hex_list

In [ ]:
hex(4043927944)

In [ ]:
from capstone import *
import re
# Machine code as hex bytes (as an example)

def get_lea_magic_number(hex_array: list)->str:
    # Dictionary to map registers to hex numbers
    register_to_hex = {
        "r8": "40",
        "r9": "41",
        "r10": "42",
        "r11": "43",
        "r12": "44",
        "r13": "45",
        "r14": "46",
        "r15": "47",
    }

    hex_str = ''.join(hex_array)
    print(hex_str)
    # Convert the hex string to bytes
    shellcode = bytes.fromhex(hex_str)
    # The first argument is the architecture type, CS_ARCH_X86 for x86.
    # The second argument is the mode, CS_MODE_64 for 64-bit mode.
    md = Cs(CS_ARCH_X86, CS_MODE_64)

    # Disassemble the code
    for i in md.disasm(shellcode, 0x1000):
        print("0x%x:\t%s\t%s" %(i.address, i.mnemonic ,i.op_str))
        op_str = i.op_str

        # Find all occurrences of "r" followed by a number
        matches = re.findall(r'r\d+', op_str)
        if matches:
            # Return the hex value corresponding to the register
            register = matches[-1]  # assuming you want the first match
            if register in register_to_hex:
                return register_to_hex[register]
        else:
            raise Exception("No register found in operation")

    raise Exception("No lea magic number found")
get_lea_magic_number(['41', '0F', 'BE', '71', '10'])


In [ ]:
import struct
import time

import pymem
from pymem.pattern import pattern_scan_all


class MemoryInjector:
    def __init__(self, target_process):
        self.target_process = target_process
        self.pm = pymem.Pymem(target_process)
        self.aob_dict = {
            "r8+10": {"aob": "41 0F BE 48 10 45 0F", "offset": 0},
            "r9+10": {"aob": "18 41 0F BE 71 10 45", "offset": 1},
            "r10+10": {"aob": "41 0F BE 72 10 45 0F B6", "offset": 0},
            "r11+10": {"aob": "45 0F BE 4B 10 41 0F B6", "offset": 0},
            "r8+10_2": {"aob": "41 0F BE 78 10 45", "offset": 0},
            "r10+10_2": {"aob": "41 0F BE 5A 10", "offset": 0},
            "r10+10_3": {"aob": "45 0F BE 4A 10 45", "offset": 0},
            # Add more keys here as needed
        }
        # Define byte sequences for lea and movsx instructions
        self.lea_dict = {
            "r9+10": ["41", "8D", "41", "10"],
            "r10+10": ["41", "8D", "42", "10"],
            "r10+10_2": ["41", "8D", "42", "10"],  # 都是一样的 _2
            "r10+10_3": ["41", "8D", "42", "10"],  # 
            "r11+10": ["41", "8D", "43", "10"],
            "r8+10": ["41", "8D", "40", "10"],
            "r8+10_2": ["41", "8D", "40", "10"]
            # Add more keys here as needed
        }
        self.movsx_dict = {
            "r9+10": ["41", "0F", "BE", "71", "10"],
            "r10+10": ["41", "0F", "BE", "72", "10"],
            "r10+10_2": ["41", "0F", "BE", "5A", "10"],  # 这个和特征码一样
            "r10+10_3": ["45", "0F", "BE", "4A", "10"],  # 这个和特征码一样
            "r11+10": ["45", "0F", "BE", "4B", "10"],
            "r8+10": ["41", "0F", "BE", "48", "10"],
            "r8+10_2": ["41", "0F", "BE", "78", "10"]
            # Add more keys here as needed
        }
        self.real_aob_offset = None
        self.aob_hex_list=[]
        self.aob_address = self.aob_scan()#这个要放在aob_hex_list后面
        self.TR = pymem.memory.allocate_memory(self.pm.process_handle, 4)
        self.newmem = None

    def aob_scan(self):
            aob_address_list = pattern_scan_all(
                handle=self.pm.process_handle,
                pattern=b'\\x0F\\xBE.\\x10.\\x0F\\xB6.\\x12',#.\\x8B.\\x24\\x45\\x0F\\xB6.\\x11',#pattern=bytes.fromhex(self.aob_dict[i]['aob']),
                return_multiple=True,
            )
            print('aob_address_list:',aob_address_list)
            if len(aob_address_list) >= 1:
                print(f"{i} try, aob_address_list offset -1: {aob_address_list} Not unique")
                aob_address=aob_address_list[0]-1
                self.aob_hex_list=[hex(b)[2:].zfill(2).upper() for b in pm.read_bytes(aob_address, 5)] # ['45', '8B', '55', '18', '45']
                print('aob_hex_list:',self.aob_hex_list,aob_address)
                return aob_address
            else:
                raise Exception("No aob found")
                
        

    @staticmethod
    def format_address(address, length=8):
        reversed_bytes = address.to_bytes(length, byteorder="little")
        reversed_formatted = " ".join(f"{b:02X}" for b in reversed_bytes)
        return reversed_formatted

    @staticmethod
    def to_signed_32_bit_le(i):
        return struct.pack("<i", i)

    # Other methods omitted for brevity
    @staticmethod
    def calculate_jmp_offset(jmp_addr, target_addr):
        offset = target_addr - (jmp_addr + 5)
        return offset
    
    def get_lea_magic_number(self)->str:
        # Dictionary to map registers to hex numbers
        register_to_hex = {
            "r8": "40",
            "r9": "41",
            "r10": "42",
            "r11": "43",
            "r12": "44",
            "r13": "45",
            "r14": "46",
            "r15": "47",
        }

        hex_str = ''.join(self.aob_hex_list)
        print(hex_str)
        # Convert the hex string to bytes
        shellcode = bytes.fromhex(hex_str)
        # The first argument is the architecture type, CS_ARCH_X86 for x86.
        # The second argument is the mode, CS_MODE_64 for 64-bit mode.
        md = Cs(CS_ARCH_X86, CS_MODE_64)

        # Disassemble the code
        for i in md.disasm(shellcode, 0x1000):
            print("0x%x:\t%s\t%s" %(i.address, i.mnemonic ,i.op_str))
            op_str = i.op_str

            # Find all occurrences of "r" followed by a number
            matches = re.findall(r'r\d+', op_str)
            if matches:
                # Return the hex value corresponding to the register
                register = matches[-1]  # assuming you want the first match
                if register in register_to_hex:
                    return register_to_hex[register]
            else:
                raise Exception("No register found in operation")

        raise Exception("No lea magic number found")
                

    def inject_memory(self):
        # Existing code before this...
        self.TR_address_reversed_formatted = self.format_address(self.TR)
        # Add your specific offsets here
        print('self.aob_hex_list:',self.aob_hex_list)
        lea_magic_number=self.get_lea_magic_number()
        lea_eax_rx_x = ["41", "8D",lea_magic_number , "10"]
        #movsx_esi_byte_ptr_rx_x = self.movsx_dict[self.real_aob_offset]
        push_rax = ["50"]
        move_TR_eax = ["A3"] + list(self.TR_address_reversed_formatted.split(" "))
        pop_rax = ["58"]
        jmp_place = ["E9"]
        part_head_combine = push_rax + lea_eax_rx_x + move_TR_eax + pop_rax + self.aob_hex_list
        shell_code_len = len(part_head_combine) + 1 + 4

        # Allocate new memory
        self.newmem = pymem.memory.allocate_memory(
            self.pm.process_handle, shell_code_len
        )

        # Calculate jmp offset 1 (newmem to aob_address)
        jmp_addr_1 = len(part_head_combine)+self.newmem
        target_addr_1 = self.aob_address+5
        offset_1 = self.calculate_jmp_offset(jmp_addr_1, target_addr_1)
        print('offset_1',offset_1)
        offset_1_le = self.to_signed_32_bit_le(offset_1)
        print('offset_1_le',offset_1_le)

        # Create shell code
        hex_values = (
            part_head_combine
            + jmp_place
            + [offset_1_le[0:1],offset_1_le[1:2],offset_1_le[2:3],offset_1_le[3:4]]
        )
        print('newmem hex_values',hex_values)
        inject_hex_shellcode = self.convert_hex_values_to_bytes(hex_values)

        # Write shell code to new memory
        result1 = self.pm.write_bytes(self.newmem, inject_hex_shellcode, len(inject_hex_shellcode))

        # Calculate jmp offset 2 (aob_address to newmem)
        jmp_addr_2 = self.aob_address
        target_addr_2 = self.newmem
        offset_2 = self.calculate_jmp_offset(jmp_addr_2, target_addr_2)
        offset_2_le = self.to_signed_32_bit_le(offset_2)

        # Create JMP shell code to inject into aob_address
        inject_hex = jmp_place + [offset_2_le[0:1],offset_2_le[1:2],offset_2_le[2:3],offset_2_le[3:4]]
        inject_hex_shellcode = self.convert_hex_values_to_bytes(inject_hex)

        # Write JMP to aob_address
        result2 = self.pm.write_bytes(self.aob_address, inject_hex_shellcode, len(inject_hex_shellcode))

        return result1, result2

    def convert_hex_values_to_bytes(self, hex_values):
        byte_values = []
        for hv in hex_values:
            if isinstance(hv, str):  # if hv is a hex string
                byte_values.append(int(hv, 16))
            elif isinstance(hv, bytes):  # if hv is a byte object
                byte_values.append(int.from_bytes(hv, "little"))
        return bytes(byte_values)
    
    def read_data(self):
        while True:
            data = self.pm.read_bytes(self.TR, 4)
            print(data)
            value = int.from_bytes(data, byteorder="little")
            print(value)
            x_address = value - 4

            data = self.pm.read_bytes(x_address, 10)
            print('x_y_map_direction',data)

            # Convert the first 2 bytes to an integer
            x_coords = int.from_bytes(data[:2], byteorder='little')  # Use 'big' if the byteorder is big endian

            print('x_coords',x_coords)

            # Convert the first 2 bytes to an integer
            y_coords = int.from_bytes(data[2:4], byteorder='little')  # Use 'big' if the byteorder is big endian

            print('y_coords',y_coords)

            map_number = int.from_bytes(data[4:6], byteorder='little')  # Use 'big' if the byteorder is big endian
            print('map_number',map_number)  # Output: 12081

            face_dir = hex(data[9])[-1]
            print('face_dir',face_dir)

            time.sleep(1)




In [ ]:
injector = MemoryInjector("javaw.exe")

In [ ]:
hex(2291495580872)

In [ ]:
print('injector.aob_address',injector.aob_address,hex(injector.aob_address))
print('TR address',injector.TR,hex(injector.TR))

In [ ]:
injector.inject_memory()

In [ ]:
print('injector.newmem',injector.newmem,hex(injector.newmem))

In [ ]:
data = injector.pm.read_bytes(injector.TR, 4)
print(data)
value = int.from_bytes(data, byteorder="little")
print(value)
x_address = value - 4

data = injector.pm.read_bytes(x_address, 10)

In [ ]:

injector.read_data()

In [ ]:
import struct

import pymem
from pymem.pattern import pattern_scan_all


class MemoryInjector:
    def __init__(self, target_process):
        self.target_process = target_process
        self.pm = pymem.Pymem(target_process)
        self.aob_dict = {
            "r8+10": {"aob": "41 0F BE 48 10 45 0F", "offset": 0},
            "r9+10": {"aob": "18 41 0F BE 71 10 45", "offset": 1},
            "r10+10": {"aob": "41 0F BE 72 10 45 0F B6", "offset": 0},
            "r11+10": {"aob": "45 0F BE 4B 10 41 0F B6", "offset": 0},
            "r8+10_2": {"aob": "41 0F BE 78 10 45", "offset": 0},
            "r10+10_2": {"aob": "41 0F BE 5A 10", "offset": 0},
            # Add more keys here as needed
        }
        # Define byte sequences for lea and movsx instructions
        self.lea_dict = {
            "r9+10": ["41", "8D", "41", "10"],
            "r10+10": ["41", "8D", "42", "10"],
            "r10+10_2": ["41", "8D", "42", "10"],  # 都是一样的 _2
            "r11+10": ["41", "8D", "43", "10"],
            "r8+10": ["41", "8D", "40", "10"],
            "r8+10_2": ["41", "8D", "40", "10"]
            # Add more keys here as needed
        }
        self.movsx_dict = {
            "r9+10": ["41", "0F", "BE", "71", "10"],
            "r10+10": ["41", "0F", "BE", "72", "10"],
            "r10+10_2": ["41", "0F", "BE", "5A", "10"],  # 这个和特征码一样
            "r11+10": ["45", "0F", "BE", "4B", "10"],
            "r8+10": ["41", "0F", "BE", "48", "10"],
            "r8+10_2": ["41", "0F", "BE", "78", "10"]
            # Add more keys here as needed
        }
        self.real_aob_offset = None
        self.aob_address = self.aob_scan()
        self.TR = pymem.memory.allocate_memory(self.pm.process_handle, 4)
        self.newmem = None

    def aob_scan(self):
        for i in self.aob_dict:
            aob_address_list = pattern_scan_all(
                handle=self.pm.process_handle,
                pattern=bytes.fromhex(self.aob_dict[i]["aob"]),
                return_multiple=True,
            )
            if len(aob_address_list) == 1:
                self.real_aob_offset = i
                real_aob_address = aob_address_list[0] + self.aob_dict[i]["offset"]
                print(
                    f"AOB Found!!! {i}, value is {self.aob_dict[i]['aob']}",
                    "aob_address",
                    i,
                    real_aob_address,
                )
                return real_aob_address
            elif len(aob_address_list) > 1:
                print(
                    f"key is {i}, value is {self.aob_dict[i]['aob']}",
                    "aob_address",
                    i,
                    aob_address_list,
                    "not unique",
                )
                continue
            else:
                print(
                    f"key is {i}, value is {self.aob_dict[i]['aob']}",
                    "aob_address",
                    i,
                    "not found",
                )
                continue
        return None

    @staticmethod
    def format_address(address, length=8):
        reversed_bytes = address.to_bytes(length, byteorder="little")
        reversed_formatted = " ".join(f"{b:02X}" for b in reversed_bytes)
        return reversed_formatted

    @staticmethod
    def to_signed_32_bit_le(i):
        return struct.pack("<i", i)

    # Other methods omitted for brevity
    @staticmethod
    def calculate_jmp_offset(jmp_addr, target_addr):
        offset = target_addr - (jmp_addr + 5)
        return offset

    def inject_memory(self):
        # Existing code before this...
        self.TR_address_reversed_formatted = self.format_address(self.TR)
        # Add your specific offsets here
        lea_eax_rx_x = self.lea_dict[self.real_aob_offset]
        movsx_esi_byte_ptr_rx_x = self.movsx_dict[self.real_aob_offset]
        push_rax = ["50"]
        move_TR_eax = ["A3"] + list(self.TR_address_reversed_formatted.split(" "))
        pop_rax = ["58"]
        jmp_place = ["E9"]
        part_head_combine = (
            push_rax + lea_eax_rx_x + move_TR_eax + pop_rax + movsx_esi_byte_ptr_rx_x
        )
        shell_code_len = len(part_head_combine) + 1 + 4

        # Allocate new memory
        self.newmem = pymem.memory.allocate_memory(
            self.pm.process_handle, shell_code_len
        )

        # Calculate jmp offset 1 (newmem to aob_address)
        jmp_addr_1 = len(part_head_combine) + self.newmem
        target_addr_1 = self.aob_address + 5
        offset_1 = self.calculate_jmp_offset(jmp_addr_1, target_addr_1)
        # print("offset_1", offset_1)
        offset_1_le = self.to_signed_32_bit_le(offset_1)
        # print("offset_1_le", offset_1_le)

        # Create shell code
        hex_values = (
            part_head_combine
            + jmp_place
            + [offset_1_le[0:1], offset_1_le[1:2], offset_1_le[2:3], offset_1_le[3:4]]
        )
        # print("newmem hex_values", hex_values)
        inject_hex_shellcode = self.convert_hex_values_to_bytes(hex_values)

        # Write shell code to new memory
        result1 = self.pm.write_bytes(
            self.newmem, inject_hex_shellcode, len(inject_hex_shellcode)
        )

        # Calculate jmp offset 2 (aob_address to newmem)
        jmp_addr_2 = self.aob_address
        target_addr_2 = self.newmem
        offset_2 = self.calculate_jmp_offset(jmp_addr_2, target_addr_2)
        offset_2_le = self.to_signed_32_bit_le(offset_2)

        # Create JMP shell code to inject into aob_address
        inject_hex = jmp_place + [
            offset_2_le[0:1],
            offset_2_le[1:2],
            offset_2_le[2:3],
            offset_2_le[3:4],
        ]
        inject_hex_shellcode = self.convert_hex_values_to_bytes(inject_hex)

        # Write JMP to aob_address
        result2 = self.pm.write_bytes(
            self.aob_address, inject_hex_shellcode, len(inject_hex_shellcode)
        )

        return result1, result2

    def convert_hex_values_to_bytes(self, hex_values):
        byte_values = []
        for hv in hex_values:
            if isinstance(hv, str):  # if hv is a hex string
                byte_values.append(int(hv, 16))
            elif isinstance(hv, bytes):  # if hv is a byte object
                byte_values.append(int.from_bytes(hv, "little"))
        return bytes(byte_values)

    def print_and_convert_bytes(self, data, start, end):
        converted = int.from_bytes(data[start:end], byteorder="little")
        # print(converted)
        return converted

    def read_data(self):
        data = self.pm.read_bytes(self.TR, 4)
        # print(data)
        value = int.from_bytes(data, byteorder="little")
        # print(value)
        x_address = value - 4-80 
        data = self.pm.read_bytes(x_address, 10+80)
        # print("x_y_map_direction", data)
        self.x_coords = self.print_and_convert_bytes(data, 0+80, 2+80)
        self.y_coords = self.print_and_convert_bytes(data, 2+80, 4+80)
        self.map_number = self.print_and_convert_bytes(data, 4+80, 6+80)
        self.face_dir = hex(data[9+80])[-1]
        self.transport=self.print_and_convert_bytes(data, 0, 2)
        data_dict = {
            "x_coords": self.x_coords,
            "y_coords": self.y_coords,
            "map_number": self.map_number,
            "face_dir": self.face_dir,
        }
        print(data_dict)
        # print("face_dir", self.face_dir)


In [ ]:
injector = MemoryInjector("javaw.exe")

In [ ]:
print('injector.aob_address',injector.aob_address,hex(injector.aob_address))
print('TR address',injector.TR,hex(injector.TR))

In [ ]:
injector.inject_memory()

In [ ]:
print('injector.newmem',injector.newmem,hex(injector.newmem))

In [ ]:
injector.read_data()

In [ ]:
import time
while True:
  data = injector.pm.read_bytes(injector.TR, 4)
  # print(data)
  value = int.from_bytes(data, byteorder="little")
  # print(value)
  x_address = value - 4-80 
  #print('x_address',x_address,hex(x_address))
  data = injector.pm.read_bytes(x_address, 10+80)
  # print("x_y_map_direction", data)
  injector.x_coords = injector.print_and_convert_bytes(data, 0+80, 2+80)
  injector.y_coords = injector.print_and_convert_bytes(data, 2+80, 4+80)
  injector.map_number = injector.print_and_convert_bytes(data, 4+80, 5+80)
  injector.face_dir = hex(data[9+80])[-1]
  injector.transport=injector.print_and_convert_bytes(data, 0, 2)
  data_dict = {
      "x_coords": injector.x_coords,
      "y_coords": injector.y_coords,
      "map_number": (int(data[4+80]),data[4+80+2],data[4+80+1]),
      "face_dir": injector.face_dir,
      'transport':injector.transport,
  }
  print(data_dict)
  # print("face_dir", self.face_dir)
  time.sleep(0.5)

In [ ]:
data = injector.pm.read_bytes(injector.TR, 4)
# print(data)
value = int.from_bytes(data, byteorder="little")


In [ ]:
import curses
import time

def main(stdscr):
    # Clear screen
    stdscr.clear()

    i = 0
    while True:
        # This will erase the existing line and refresh the screen
        stdscr.erase()

        # Now print at (0, 0) with a counter that increments every second
        stdscr.addstr(0, 0, f"Counter: {i}")

        # Refresh the screen
        stdscr.refresh()

        # Wait for 1 second
        time.sleep(1)
        i += 1

# Run the program
curses.wrapper(main)


In [ ]:
hex(2053499191296)

In [ ]:

shellcode = b'\x01\x00\x00\x00\x00\x00\x00\x00\x9eB\x03 \x00\x00\x00\x00\xe0\xca\xaa\xf3X\xdb\x8d\xf3\x00\x00\x00\x00\xa8j\x8f\xf3\x10\xfe\xd6\xea\x00\x00\x00\x00\xe8j\x8f\xf3\x08k\x8f\xf3\xc0]r\xf3(k\x8f\xf3\x00\x00\x00\x00'

single_float_bytes = shellcode[14:18]  # 获取单精度浮点数的4个字节
single_float = struct.unpack('<f', single_float_bytes)[0]  # 将字节转换为单精度浮点数

print(single_float)

In [ ]:
import json
import os
import re
import struct
import time

import pymem
from capstone import *
from pymem.pattern import pattern_scan_all


def format_address(address, length=8):
    reversed_bytes = address.to_bytes(length, byteorder="little")
    reversed_formatted = " ".join(f"{b:02X}" for b in reversed_bytes)
    return reversed_formatted


def to_signed_32_bit_le(i):
    return struct.pack("<i", i)


def calculate_jmp_offset(jmp_addr, target_addr):
    offset = target_addr - (jmp_addr + 5)
    return offset


def split_bytes_to_int(data, start, end):
    converted = int.from_bytes(data[start:end], byteorder="little")
    return converted


class MemoryInjector:
    def __init__(self, name, pattern, offset, json_file_path, aob_hex_list_len):
        self.target_process = "javaw.exe"
        self.pattern = pattern
        self.process_name = name
        self.offset = offset
        self.pm = pymem.Pymem(self.target_process)
        self.memory_info_dict = {}

        # Adding a path to your json file
        self.json_file_path = json_file_path
        self.aob_hex_list_len = aob_hex_list_len  # 注入那条指令的长度
        self.aob_address_list = []
        self.aob_hex_list = []
        self.aob_address = 0
        self.TR = None
        self.newmem = None
        # Try to read the stored values from the json file
        if os.path.exists(self.json_file_path):
            with open(self.json_file_path, "r") as json_file:
                data = json.load(json_file)
                self.aob_address = data.get("aob_address", 0)
                self.TR = data.get("TR", None)
                self.newmem = data.get("newmem", None)
                print(
                    f"Read data from json file. aob_address: {self.aob_address}, TR: {self.TR}, newmem: {self.newmem}"
                )

            # Try to read data
            try:
                self.read_data()
                print(
                    f"{self.process_name} Reading data succeeded with stored addresses. aob_address: {self.aob_address}, TR: {self.TR}, newmem: {self.newmem}"
                )
            except Exception as e:
                print(
                    f"{self.process_name} Reading data failed with stored addresses, starting normal scanning and injection process:",
                    e,
                )
                self.aob_address = self.aob_scan()
                self.try_the_aob()
        else:
            self.aob_address = self.aob_scan()
            self.try_the_aob()

    def aob_scan(self):
        while True:
            self.aob_address_list = pattern_scan_all(
                handle=self.pm.process_handle,
                pattern=self.pattern,  #  b"\\x45\\x8B\\x9A\\x98\\x00\\x00\\x00\\x45\\x8B.\\xAC\\x00\\x00\\x00\\x4D\\x8B\\xD3"
                return_multiple=True,
            )
            print("aob_address_list:", self.aob_address_list)
            if len(self.aob_address_list) >= 1:
                print(f"{self.process_name}aob_address_list:", self.aob_address_list)
                return
            if len(self.aob_address_list) < 1:
                print(f"waiting for AOB {self.process_name} appear, retrying...")
                time.sleep(1)

    def try_the_aob(self):
        print("Trying the aob")
        print("aob_address_list:", self.aob_address_list)
        for i in self.aob_address_list:
            self.aob_address = i + self.offset  # 必定要偏移
            self.aob_hex_list = [
                hex(b)[2:].zfill(2).upper()
                for b in self.pm.read_bytes(self.aob_address, self.aob_hex_list_len)
            ]
            print("aob_hex_list:", self.aob_hex_list, self.aob_address)

            try:
                self.inject_memory()
                time.sleep(1)
                self.read_data()
                with open(self.json_file_path, "w") as json_file:
                    json.dump(
                        {
                            "aob_address": self.aob_address,
                            "TR": self.TR,
                            "newmem": self.newmem,
                            "timestamp": time.time(),  # current timestamp
                        },
                        json_file,
                    )

                print(
                    f"Injected {self.process_name}, aob_address: {self.aob_address}, TR: {self.TR}, newmem: {self.newmem}, aob_hex_list: {self.aob_hex_list}"
                )
            except Exception as e:
                print(e)
                continue

    def inject_memory(self):
        # Existing code before this...
        self.TR = pymem.memory.allocate_memory(self.pm.process_handle, 4)
        self.TR_address_reversed_formatted = format_address(self.TR)
        push_rax_lea_eax_rx_x = ["50", "41", "8D", "82", "98", "00", "00", "00"]
        move_TR_eax = ["A3"] + list(self.TR_address_reversed_formatted.split(" "))
        pop_rax = ["58"]
        jmp_place = ["E9"]
        part_head_combine = (
            push_rax_lea_eax_rx_x + move_TR_eax + pop_rax + self.aob_hex_list
        )
        shell_code_len = len(part_head_combine) + 5  # 5:jmp_place + offset_1_le

        # Allocate new memory
        self.newmem = pymem.memory.allocate_memory(
            self.pm.process_handle, shell_code_len
        )

        # Calculate jmp (newmem to aob_address)
        jmp_addr_1 = len(part_head_combine) + self.newmem
        target_addr_1 = self.aob_address + self.aob_hex_list_len
        offset_1 = calculate_jmp_offset(jmp_addr_1, target_addr_1)
        print("offset_1", offset_1)
        offset_1_le = to_signed_32_bit_le(offset_1)
        print("offset_1_le", offset_1_le)

        # Create shell code
        hex_values = (
            part_head_combine
            + jmp_place
            + [offset_1_le[0:1], offset_1_le[1:2], offset_1_le[2:3], offset_1_le[3:4]]
        )
        print("newmem hex_values", hex_values)
        inject_hex_shellcode = self.convert_hex_values_to_bytes(hex_values)

        # Write shell code to new memory
        result1 = self.pm.write_bytes(
            self.newmem, inject_hex_shellcode, len(inject_hex_shellcode)
        )

        # Calculate jmp offset 2 (aob_address to newmem)
        jmp_addr_2 = self.aob_address
        target_addr_2 = self.newmem
        offset_2 = calculate_jmp_offset(jmp_addr_2, target_addr_2)
        offset_2_le = to_signed_32_bit_le(offset_2)

        # Create JMP shell code to inject into aob_address
        extra_nop = None
        if self.aob_hex_list_len - 5 == 2:
            extra_nop = ["66", "90"]
        elif self.aob_hex_list_len - 5 == 1:
            extra_nop = ["90"]
        elif self.aob_hex_list_len == 5:
            pass
        else:
            raise Exception("aob_hex_list_len must be 6 or 7")

        inject_hex = (
            jmp_place
            + [
                offset_2_le[0:1],
                offset_2_le[1:2],
                offset_2_le[2:3],
                offset_2_le[3:4],
            ]
            + extra_nop
        )
        inject_hex_shellcode = self.convert_hex_values_to_bytes(inject_hex)

        # Write JMP to aob_address
        result2 = self.pm.write_bytes(
            self.aob_address, inject_hex_shellcode, len(inject_hex_shellcode)
        )

        return result1, result2

    def convert_hex_values_to_bytes(self, hex_values):
        byte_values = []
        for hv in hex_values:
            if isinstance(hv, str):  # if hv is a hex string
                byte_values.append(int(hv, 16))
            elif isinstance(hv, bytes):  # if hv is a byte object
                byte_values.append(int.from_bytes(hv, "little"))
        return bytes(byte_values)

    def read_data(self):
        pass
        # data = self.pm.read_bytes(self.TR, 4)
        # value = int.from_bytes(data, byteorder="little")
        # x_address = value - 4 - 80
        # # print("x_address", x_address, hex(x_address))
        # data = self.pm.read_bytes(x_address, 10 + 80)
        # # print("x_y_map_direction", data)
        # self.x_coords = split_bytes_to_int(data, 0 + 80, 2 + 80)
        # self.y_coords = split_bytes_to_int(data, 2 + 80, 4 + 80)
        # self.map_number = split_bytes_to_int(data, 4 + 80, 5 + 80)
        # self.face_dir = hex(data[9 + 80])[-1]
        # self.transport = split_bytes_to_int(data, 0, 2)
        # self.memory_info_dict = {
        #     "x_coords": self.x_coords,
        #     "y_coords": self.y_coords,
        #     "map_number": (int(data[4 + 80]), data[4 + 80 + 2], data[4 + 80 + 1]),
        #     "face_dir": self.face_dir,
        #     "transport": self.transport,
        # }
        # if self.memory_info_dict["map_number"] == (0, 0, 0):
        #     raise Exception("Map number is 0, 0, 0")
        # return self.memory_info_dict
        # print(self.memory_info_dict)

In [ ]:
injector = MemoryInjector(
    name="Battle_Memory_Injector",
    pattern=b"\\x45\\x8B\\x9A\\x98\\x00\\x00\\x00\\x45\\x8B.\\xAC\\x00\\x00\\x00\\x4D\\x8B\\xD3",
    offset=0,
    json_file_path="battle_memory_injector.json",
    aob_hex_list_len=7,
)

In [11]:
target_process = "javaw.exe"
pm = pymem.Pymem(target_process)

In [12]:
data = pm.read_bytes('1779162939392', 4)
# value = int.from_bytes(data, byteorder="little")
# target_value = pm.read_bytes(value, 28)
# print("target_value", target_value, hex(int.from_bytes(data, byteorder="little")))
# value = int.from_bytes(data, byteorder="little")
# print("value", value)
# target_value = pm.read_bytes(data, 28)
# print("target_value", target_value)

TypeError: cannot be converted to pointer